In [1]:
import os

list_file = sorted(list(os.walk('./file/4.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['crawler_1.xlsx', 'crawler_2.xlsx']

In [2]:
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./file/4.part/{file}',
                       header=0,
                       dtype=str).fillna('')

input_['Product No'] = input_['Product No'].astype(int)
input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Part Number'].tolist()

总数量：1


['TCK265A']

In [5]:
a = 0

a

0

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [7]:
import requests

import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

b = 0
while True:
    b += 1
    print(b)
    try:
        resp = requests.get(f'''https://gates.scene7.com/is/image/{input_.loc[a, 'Asset']}?req=set,json,UTF-8&labelkey=label&handler=s7classics7sdkJSONResponse''',
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(10, 10))

        if resp.status_code == 200:
            if 'Error while processing Fvctx image' in resp.text and 'Error while processing Fvctx image' in resp.text:
                break
            
            json_ = json.loads(resp.text.split('s7classics7sdkJSONResponse(')[1].split(',"");')[0])
            if json_:
                break
    except KeyboardInterrupt:
        break
    except:
        continue

resp

1
2
3
4
5
6
7


<Response [200]>

In [8]:
if 'Error while processing Fvctx image' in resp.text and 'Error while processing Fvctx image' in resp.text:
    dict_src = {}
else:
    dict_src, list_src = {}, [dict_['s']['n'].strip() if 's' in dict_ else dict_['set']['n'] for dict_ in json_['set']['item']] if type(json_['set']['item']) == list else [json_['set']['item']['s']['n'].strip() if 's' in json_['set']['item'] else json_['set']['item']['i']['n'].strip()]
    for i, src in zip(range(len(list_src)), list_src):
        dict_src[str(i)] = f'''https://gates.scene7.com/is/image/{src}'''

dict_src

{'0': 'https://gates.scene7.com/is/image/gates/tck265a-1-kit-a',
 '1': 'https://gates.scene7.com/is/image/gates/ss-tck265a',
 '2': 'https://gates.scene7.com/is/image/gates/ss-t43131',
 '3': 'https://gates.scene7.com/is/image/gates/ss-t42121',
 '4': 'https://gates.scene7.com/is/image/gates/ss-t265'}

In [9]:
len(dict_src) == 0

False

In [10]:
df_temp = pd.DataFrame([{'Product No': input_.loc[a, 'Product No'],
                         'Product Name': input_.loc[a, 'Product Name'],
                         'Product Number': input_.loc[a, 'Product Number'],
                         'No': input_.loc[a, 'No'],
                         'Part Number': input_.loc[a, 'Part Number'],
                         'Type': input_.loc[a, 'Type'],
                         'SKU': input_.loc[a, 'SKU'],
                         'UPC': input_.loc[a, 'UPC'],
                         'Title': input_.loc[a, 'Title'],
                         'Picture': '',
                         'Url': input_.loc[a, 'Url'],
                         'Json_Src': json.dumps(dict_src),
                         'Json_Information': input_.loc[a, 'Json_Information'],
                         'Part Code': input_.loc[a, 'Part Code'],
                         'Asset': input_.loc[a, 'Asset']}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,Product No,Product Name,Product Number,No,Part Number,Type,SKU,UPC,Title,Picture,Url,Json_Src,Json_Information,Part Code,Asset
0,1,PowerGrip™ Timing Kits,7782-000000-000001,107,TCK265A,KitVariantProduct,77820317,770075351726,TCK265A TIMING COMPONENT KIT,,https://www.gates.com/us/en/search.p.7782-0000...,"{""0"": ""https://gates.scene7.com/is/image/gates...","{""0"": {""Instruction Sheet Number"": ""0098-1572,...",7782-00317,gates/mm-100876-tck265a


In [11]:
crawler_status = 'ok'

crawler_status

'ok'

In [12]:
if crawler_status == 'error':
    output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

output_error

""


In [13]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Product No', 'No'],
                                                ascending=[True, True],
                                                ignore_index=True)
    output_correct.to_excel('./test_part.xlsx', index=False)

output_correct

,Product No,Product Name,Product Number,No,Part Number,Type,SKU,UPC,Title,Picture,Url,Json_Src,Json_Information,Part Code,Asset
0,1,PowerGrip™ Timing Kits,7782-000000-000001,107,TCK265A,KitVariantProduct,77820317,770075351726,TCK265A TIMING COMPONENT KIT,,https://www.gates.com/us/en/search.p.7782-0000...,"{""0"": ""https://gates.scene7.com/is/image/gates...","{""0"": {""Instruction Sheet Number"": ""0098-1572,...",7782-00317,gates/mm-100876-tck265a


In [14]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['Product No', 'No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_part.xlsx', index=False)

output_error

""
